In [25]:
import pandas as pd
import datetime
import re
import time

In [26]:
def remove_str(row):
    x = row.Comments2
    try:
        x = x.replace(re.search('Originally posted .*', x)[0], '').strip()
    except:
        pass
    try:
        x = x.replace(re.search('Release Date:.*', x)[0], '').strip()
    except:
        pass
    row.Comments2 = x
    return row

Previous Year's Number of Post

In [27]:
def prev_year(date_temp):
    return df1[(((df1['Date Posted'] - date_temp).astype('timedelta64[h]')/24) >= -365) & (((df1['Date Posted'] - date_temp).astype('timedelta64[h]')/24) <=0)].shape[0]

Total Post Related to Earnings

In [28]:
def earning_post(filtered_df, keyword):
    if filtered_df.shape[0] == 0:
        return 0
    def func_inner(row):      
        for each_word in keyword:
            if each_word.lower() in str(row.Comments2).lower():
                return 1
        return 0
    return sum(filtered_df.apply(func_inner, axis='columns'))

Remove Unwanted Lines from Comments

In [29]:
# filtered_df = filtered_df.apply(remove_str, axis='columns')

Row containing at least one negative words

In [30]:
def row_containing_neg_words(filtered_df, keyword):
    if filtered_df.shape[0] == 0:
        return 0
    def func_inner(row):      
        for each_word in keyword:
            if each_word in str(row.Comments2):
                return 1
        return 0
    return sum(filtered_df.apply(func_inner, axis='columns'))

Number of Positive Words Mentioned

In [31]:
def positive_words(filtered_df, keyword):
    if filtered_df.shape[0] == 0:
        return 0
    if filtered_df.shape[0] == 0:
        return 0
    def func2(row):
        word_count = 0
        for each in keyword:
            word_count += len(re.findall(each, str(row.Comments2)))
        return word_count
    return sum(filtered_df.apply(func2, axis='columns'))

Number of Negative Words Mentioned

In [32]:
def criticism_words(filtered_df, keyword):
    if filtered_df.shape[0] == 0:
        return 0
    def func2(row):
        word_count = 0
        for each in keyword:
            word_count += len(re.findall(each.lower(), str(row.Comments2)))
        return word_count
    return sum(filtered_df.apply(func2, axis='columns'))

Total Number of Words

In [33]:
def words_length(filtered_df):
    if filtered_df.shape[0] == 0:
        return 0
    def func3(row):
        return len(str(row.Comments2).split())
    return sum(filtered_df.apply(func3, axis='columns'))

Sentiment

In [34]:
def sentiment(df):
    df['Sentiment'] = df['Sentiment'].fillna('None')
    df.Sentiment = df.Sentiment.replace('', 'None')
    try:
        buy  = df.Sentiment.value_counts()['Buy']
    except:
        buy = 0
    try:
        hold = df.Sentiment.value_counts()['Hold']
    except:
        hold = 0
    try:
        sell = df.Sentiment.value_counts()['Sell']
    except:
        sell = 0
    total = df.shape[0]
    try:
        sent = (buy+hold-sell)/total
    except:
        sent = 0
         # total is zero
    return sent



Function that manage program flow

In [35]:
def process_function(row): 
    pos_keyword = ['am', 'is', 'are', 'was', 'were']
    neg_keyword = ['am', 'is', 'are', 'was', 'were']
    earning = ['hi', 'hello']
    
    # Filtering Data Frame before 365 days
    filtered_df = df1[(((df1['Date Posted'] - row['Year']).astype('timedelta64[h]')/24) >= -365) & (((df1['Date Posted'] - row['Year']).astype('timedelta64[h]')/24) <=0)]
    row['Total Post in a Financial Year'] = filtered_df.shape[0]
    row['Total post related to earnings'] = earning_post(filtered_df, earning)
    
    # Filtering Data that day, 1 day before and after
    filtered_df_3_days = df1[(abs((df1['Date Posted'] - row['Year']).astype('timedelta64[h]')/24) <= 1)]
    
    # Filtering data between 90 to 21 days
    filtered_df_90_21 = df1[(((df1['Date Posted'] - row['Year']).astype('timedelta64[h]')/24) >= -90) & (((df1['Date Posted'] - row['Year']).astype('timedelta64[h]')/24) <= -21)]
    row['Abnormal Discussions'] = filtered_df_3_days.shape[0]/filtered_df_90_21.shape[0]
    row['Total number of positive words'] = positive_words(filtered_df_3_days, pos_keyword)
    row['Criticism Comments'] = row_containing_neg_words(filtered_df_3_days, neg_keyword)
    row['Total number of negative words'] = criticism_words(filtered_df_3_days, neg_keyword)
    row['Total no. of words'] = words_length(filtered_df_3_days)
    row['Sentiment'] = sentiment(filtered_df_3_days)
    return row


Main Loop

In [36]:
import os
try:
    os.chdir(r'..\New folder')
except:
    pass
files = [i for i in os.listdir() if '.xlsx' in i]
for i in range(0, len(files), 2):
    # Read Excel File
    df1 = pd.read_excel(files[i])
    df2 = pd.read_excel(files[i+1])
    # Fill Empty Cell
    df1['Date Posted'] = df1['Date Posted'].fillna(method='ffill')
    df1 = df1.fillna('')
    #Convert to Datetime
    df1['Date Posted'] = pd.to_datetime(df1['Date Posted'])
    df2.Year = pd.to_datetime(df2.Year, format=r'%Y-%m-%d')
    df2 = df2.apply(process_function, axis='columns')
    df2.to_csv(str(i) + '.csv', index=False)